In [1]:
import pandas as pd
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from typing import List
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
instructor.patch()

In [3]:
class ConversationObject(BaseModel):
    role: str
    content: str = Field('...', description = 'if the role is `assistant`, it must convert user content into function parameter format `<functioncall> {"name", "arguments"}`')
        
class JsonObject(BaseModel):
    function_call: str
    conversations: List[ConversationObject]

In [4]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai10-aiservices-2093329748.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ""

engine = 'gpt-35-turbo'

In [5]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=1.0,
        max_tokens=max_tokens,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )
        return response
    except Exception as e:
        print(e)
        return None
    
def predict_instruct(text):
    r = openai.ChatCompletion.create(
        engine=engine,
        response_model=JsonObject,
        messages=[
            {"role": "user", "content": text},
        ],
        temperature=1.0,
        max_tokens=1024,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return r

In [6]:
function_call = {
        'name': 'UpdateRoles',
        'description': 'Fungsi ini mengemas kini peranan pengguna. Ia menerima pelbagai objek. Setiap objek hendaklah memasukkan nama peranan, alamat e-mel dan tindakan yang menentukan sama ada peranan itu perlu ditambah atau dialih keluar.',
        'parameters': {
            'type': 'object',
            'properties': {
                'role_users': {
                    'type': 'array',
                    'items': {
                        'type': 'object',
                        'properties': {
                            'rolename': {
                                'type': 'string',
                                'enum': ['role1','role2','admin'],
                                'description': 'Nama peranan yang akan dikemas kini.',
                            },
                            'email': {
                                'type': 'string',
                                'description': 'Alamat e-mel pengguna yang peranannya sedang dikemas kini.',
                            },
                            'action': {
                                'type': 'string',
                                'enum': ['tambah', 'buang'],
                                'description': "dia bertindak untuk dilakukan. 'tambah' untuk memberikan peranan kepada pengguna, 'buang' untuk mengeluarkan peranan daripada pengguna.",
                            },
                        },
                        'required': ['rolename', 'email', 'action'],
                    },
                }
            },
            'required': ['role_users'],
        },
    }

a = '{\n' + \
'  "role_users": [\n' + \
'    {\n' + \
'      "rolename": "role1",\n' + \
'      "email": "test@email",\n' + \
'      "action": "tambah"\n' + \
'    },\n' + \
'    {\n' + \
'      "rolename": "admin",\n' + \
'      "email": "mike@email",\n' + \
'      "action": "buang"\n' + \
'    }\n' + \
'  ]\n' + \
'}'

In [7]:
f = str({'name': 'UpdateRoles', 'arguments': json.dumps(json.loads(a))})

s = """
saya ada 2 orang kena edit,
1. tambah role1, email test@email
2. buang admin, email mike@email
"""

conversations = [
    {
        'role': 'user',
        'content': s.strip()
    },
    {
        'role': 'assistant',
        'content': f'<functioncall> {f}'
    }
]

e = {
    'function_call': json.dumps(function_call),
    'conversations': conversations,
}
text = f"""Generate complex Malay function call prompts inspired by the given example. If translation is challenging, use Malaysian terms for variety. Maintain the JSON format, altering only key contents. Follow instructions precisely, avoiding randomness.

Code snippet for inspiration:
```
{e}
```
"""
r = predict_instruct(text)

In [8]:
!mkdir function-call-complex

mkdir: cannot create directory ‘function-call-complex’: File exists


In [9]:
r.dict()

/tmp/ipykernel_6494/3582938815.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  r.dict()


{'function_call': '{"name":"GeneratePayment","description":"Fungsi ini mencipta bayaran untuk perkhidmatan. Ia menerima maklumat perkhidmatan dan butiran bayaran. Butiran-butiran tersebut hendaklah memasukkan jenis perkhidmatan, jumlah bayaran, dan tarikh bayaran.","parameters":{"type":"object","properties":{"service_details":{"type":"object","properties":{"service_type":{"type":"string","enum":["servis1","servis2","servis3"],"description":"Jenis perkhidmatan yang ingin dibayar."},"payment_amount":{"type":"number","description":"Jumlah bayaran yang perlu dibuat."},"payment_date":{"type":"string","format":"date","description":"Tarikh bayaran yang ditetapkan."},"required":["service_type","payment_amount","payment_date"]}}},"required":["service_details"]}}',
 'conversations': [{'role': 'user',
   'content': 'Saya perlu cipta bayaran untuk:\n1. servis1, bayaran RM200, tarikh 2023-12-31\n2. servis2, bayaran RM150, tarikh 2023-11-15'},
  {'role': 'assistant',
   'content': '<functioncall> {"

In [ ]:
from tqdm import tqdm

for i in tqdm(range(20000)):
    filename = f'function-call-complex/{i}.json'
    if os.path.exists(filename):
        continue
    
    try:
        r = predict_instruct(text)
        with open(filename, 'w') as fopen:
            json.dump(r.dict(), fopen)
    except:
        pass

  0%|                                                                                                                   | 0/20000 [00:00<?, ?it/s]/tmp/ipykernel_6494/3863016856.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  json.dump(r.dict(), fopen)
 43%|█████████████████████████████████████████████▏                                                          | 8693/20000 [12:33<17:48, 10.59it/s]